In [30]:
import requests
import pandas as pd
import random

from bs4 import BeautifulSoup

In [31]:
def scrape(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0'})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [32]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [33]:
def get_product(url):
    soup = scrape(url)

    products = soup.find('div', class_='layout__Container-sc-nb1ebc-0 NDIcW')

    attributes = {'Name':[], 'Image Link':[], 'Origin':[], 'Price':[], 'Unit Price':[]}

    for product in products:
        name = product.find('a', class_='link__Link-sc-14ymsi2-0 hSRoYD link__Link-sc-14ymsi2-0 base__Title-sc-1mnb0pd-27 base__FixedHeightTitle-sc-1mnb0pd-43 hSRoYD ifdXVr cCRJZx')
        #print(soup.select('#main > div:nth-child(2) > div > div > div:nth-child(2) > div > div > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div > a'))
        origin = product.find('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo')
        price = product.find('strong', class_='base__Price-sc-1mnb0pd-29 sc-ksdxAp fJKWWj irZyf')
        unit_price = product.find('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy')

        if price:
            attributes['Price'].append(price.text.strip())
        else:
            attributes['Price'].append(None)

        if name:
            attributes['Name'].append(name.text.strip())
        else:
            attributes['Name'].append(None)

        if origin:
            attributes['Origin'].append(origin.text.strip())
        else:
            attributes['Origin'].append(None)

        if unit_price:
            attributes['Unit Price'].append(unit_price.text.strip())
        else:
            attributes['Unit Price'].append(None)
        
        attributes['Image Link'].append(None)  

    # if image_link:
    #     attributes['Image Link'].append(image_link['href'])
    # else:
    #     attributes['Image Link'].append('N/A')
    
    return attributes

In [34]:
urls = ['https://plus.casino.fr/products?sublocationId=86df9681-98f0-424e-80aa-e335a882e6aa']

for url in urls:
    product_dict = get_product(url)
    retail_df = pd.DataFrame(product_dict)

In [35]:
retail_df.dropna(how='all', inplace=True)

In [38]:
# Output the filtered dataframe to a CSV file
retail_df.to_csv('casino.csv', index=False)